# SEQUENCE CODE for per-gaussian movement !!!!

In [1]:
%load_ext autoreload
%autoreload 2

import torch
import numpy as np

In [21]:
from thesis.data_management import SequenceManager

sequence = 93
sm = SequenceManager(sequence)
a = torch.load(
    f'/home/schlack/thesis-quantization/saved_vertex_preds/sequence_{sequence}.pt',
    weights_only=True)
print(f'n_frames: {len(sm)}, pred_frames: {a.shape[0]}')

n_frames: 124, pred_frames: 122


In [17]:
from thesis.video_utils import side_by_side

side_by_side(
    video_gt='ablations/ablations_final/with_2dgs/flame/videos/sequence_3_gt.mp4',
    video_pred='ablations/ablations_final/with_2dgs/flame/videos/sequence_3.mp4',
    output_path='ablations/ablations_final/with_2dgs/flame/videos/sequence_80_side_by_side.mp4',
)


MoviePy - Building video ablations/ablations_final/with_2dgs/flame/videos/sequence_80_side_by_side.mp4.
MoviePy - Writing audio in sequence_80_side_by_sideTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
MoviePy - Writing video ablations/ablations_final/with_2dgs/flame/videos/sequence_80_side_by_side.mp4



frame_index:  99%|█████████▉| 261/263 [00:11<00:00, 22.99it/s, now=None]/home/schlack/miniconda3/envs/thesis_quantization/lib/python3.10/site-packages/moviepy/video/io/ffmpeg_reader.py:157: UserWarning: In file ablations/ablations_final/with_2dgs/flame/videos/sequence_3_gt.mp4, 5293200 bytes wanted but 0 bytes read at frame index 262 (out of a total 263 frames), at time 8.73/8.78 sec. Using the last valid frame instead.
  warnings.warn(
/home/schlack/miniconda3/envs/thesis_quantization/lib/python3.10/site-packages/moviepy/video/io/ffmpeg_reader.py:157: UserWarning: In file ablations/ablations_final/with_2dgs/flame/videos/sequence_3.mp4, 5293200 bytes wanted but 0 bytes read at frame index 262 (out of a total 263 frames), at time 8.73/8.78 sec. Using the last valid frame instead.
  warnings.warn(


MoviePy - Done !
MoviePy - video ready ablations/ablations_final/with_2dgs/flame/videos/sequence_80_side_by_side.mp4


In [3]:
from thesis.evaluation_runner import load_compressed_array

path = 'ablations/ablations_final/with_2dgs/flame/sequence_3'
reference_path = f'{path}.npy'
bz2_path = f'{path}.bz2'
gzip_path = f'{path}.gzip'
lzma_path = f'{path}.lzma'

reference = np.load(reference_path)
reference.shape

(263, 1604, 1100, 3)

In [46]:
from thesis.gaussian_splatting.implicit_sequence_adjustment import ImplicitSequenceAdjustment

isa = ImplicitSequenceAdjustment()
a = torch.tensor(3)  #.unsqueeze(0).repeat(4)
b = torch.tensor(4)  #.unsqueeze(0).repeat(4)
isa(a, b).shape

torch.Size([5443, 3])

In [27]:
# load config
from thesis.config import load_config

config_path = "configs/single_frame.yml"
config = load_config(config_path)

TypeError: load_config() missing 1 required positional argument: 'mode'

In [ ]:
# load single frame model
from thesis.gaussian_splatting.single_frame import GaussianSplattingSingleFrame

model = GaussianSplattingSingleFrame(
    gaussian_splatting_settings=config.gaussian_splatting_settings,
    learning_rates=config.learning_rates,
)
model = model.cuda()

In [ ]:
# sanity check
params = model.splats
optimizers, schedulers = model.configure_optimizers()
splat_optimizers = {k: optimizers[i] for i, k in enumerate(model.splat_optimizer_keys)}
model.strategy.check_sanity(params, splat_optimizers)

In [ ]:
from thesis.data_management import SingleSequenceDataset
from torch.utils.data import DataLoader

train_set = SingleSequenceDataset(
    sequence=config.sequence,
    start_idx=config.frame,
    end_idx=config.frame + 1,
    n_cameras_per_frame=config.gaussian_splatting_settings.camera_batch_size)

train_loader = DataLoader(train_set, batch_size=None, shuffle=True)

In [ ]:
a = next(iter(train_loader))
batch = train_set.prepare_data(a)
rendered_images, rendered_alphas, infos = model.forward(
    intrinsics=batch.intrinsics,
    world_2_cam=batch.world_2_cam,
    cam_2_world=None,
    image_height=int(batch.image.shape[1]),
    image_width=int(batch.image.shape[2]),
    # color_correction=batch.color_correction, # TODO: fix color correction
    cur_sh_degree=None,
    se3_transform=batch.se3_transform,
)

In [ ]:
from matplotlib import pyplot as plt
from dreifus.render import project
from dreifus.matrix import Pose, Intrinsics

intrinsics = Intrinsics(batch.intrinsics[0].detach().cpu().numpy())
pose = Pose(batch.world_2_cam[0].detach().cpu().numpy())
projected = project(model.splats['means'].detach().cpu().numpy(), pose, intrinsics)
image = rendered_images[0].detach().cpu().numpy()
projected_x = projected[:, 0].round().astype(int)
projected_y = projected[:, 1].round().astype(int)
valid_x = (0 <= projected_x) & (projected_x < image.shape[1])
valid_y = (0 <= projected_y) & (projected_y < image.shape[0])
valid_xy = valid_x & valid_y
print(f"{valid_xy.sum() / projected.shape[0] * 100:.1f}% of the vertices are visible"
      f" i.e. {valid_xy.sum()} out of {projected.shape[0]}\n")
for y, x in zip(projected_y[valid_xy], projected_x[valid_xy]):
    for i in range(-3, 3):
        if not 0 <= y + i < image.shape[0]:
            continue
        for j in range(-3, 3):
            if not 0 <= x + j < image.shape[1]:
                continue
            image[y + i, x + j] = [255, 255, 255]

plt.imshow(image)
plt.show()

In [ ]:
path = '/home/schlack/master-thesis/data/Paul-audio-85/085/sequences/SEN-05-glow_eyes_sweet_girl/annotations/tracking/FLAME2023_v2/tracked_flame_params.npz'
data = np.load(path)
data['translation'][72 // 3]